In [11]:
#import libraries
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.optimizers import Adam,RMSprop

In [12]:
path='D:/Datasets/Image dataset/eye disease/eye-disease-dataset/'


In [13]:
#size of images to feed in neural network
image_shape = (100,100,3) 
#data augmentation

datagen = ImageDataGenerator(rotation_range=30, # rotate the image 30 degrees
                               width_shift_range=0.1, # Shift the pic width by a max of 10%
                               height_shift_range=0.1, # Shift the pic height by a max of 10%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.2, # Shear means cutting away part of the image (max 20%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               vertical_flip=True,
                               fill_mode='nearest',# Fill in missing pixels with the nearest filled value
                               validation_split=0.2#split data to train and test
                              )

In [14]:
#load the training data
train_generator = datagen.flow_from_directory(
    path+'Cropped_Images/',
    target_size=image_shape[0:2],
    batch_size=100,
    class_mode='categorical',
    subset='training')

Found 726 images belonging to 5 classes.


In [15]:
#load the test data
test_generator = datagen.flow_from_directory(
    path+'Cropped_Images/',
    target_size=image_shape[0:2],
    batch_size=100,
    class_mode='categorical',
    subset='validation')


Found 180 images belonging to 5 classes.


In [32]:
#create model
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = 3,input_shape=image_shape,padding='same'))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters = 32,kernel_size = 3,activation= 'relu',padding='same'))
#model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters = 64,kernel_size = 3,activation= 'relu',padding='same'))
#model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters = 64,kernel_size = 5,activation= 'relu',padding='valid'))
model.add(Conv2D(filters = 128,kernel_size = 3,activation= 'relu',padding='valid'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(5,activation = 'softmax'))


In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 16)      448       
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 96, 96, 64)        102464    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 94, 94, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 47, 128)       0         
__________

In [34]:
#compile the model
opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [35]:
#create class weights for unbalance dataset
from sklearn.utils import class_weight
y_train=train_generator.classes
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))


In [36]:
es=keras.callbacks.EarlyStopping(monitor='val_acc',mode='min')

In [37]:
#train model
results = model.fit_generator(train_generator,epochs=10,
                              steps_per_epoch=30,
                              validation_data=test_generator,
                             validation_steps=30,
                              #callbacks=[es],
                             class_weight=class_weights)


Epoch 1/10
30/30 [==============================] - 61s 2s/step - loss: 1.6302 - acc: 0.1718 - val_loss: 1.5973 - val_acc: 0.2237
Epoch 2/10
30/30 [==============================] - 57s 2s/step - loss: 1.5490 - acc: 0.2809 - val_loss: 1.5536 - val_acc: 0.2330
Epoch 3/10
30/30 [==============================] - 55s 2s/step - loss: 1.5052 - acc: 0.3094 - val_loss: 1.4780 - val_acc: 0.2911
Epoch 4/10
30/30 [==============================] - 55s 2s/step - loss: 1.4466 - acc: 0.3404 - val_loss: 1.5059 - val_acc: 0.3226
Epoch 5/10
30/30 [==============================] - 57s 2s/step - loss: 1.4150 - acc: 0.3637 - val_loss: 1.4585 - val_acc: 0.3911
Epoch 6/10
30/30 [==============================] - 58s 2s/step - loss: 1.3437 - acc: 0.3977 - val_loss: 1.5270 - val_acc: 0.3485
Epoch 7/10
30/30 [==============================] - 58s 2s/step - loss: 1.2772 - acc: 0.4076 - val_loss: 1.5433 - val_acc: 0.3485
Epoch 8/10
30/30 [==============================] - 57s 2s/step - loss: 1.2239 - acc: 0.45

In [38]:
#print classification report
y_true=test_generator.classes
class_label=list(train_generator.class_indices.keys())
from sklearn.metrics import classification_report
y_pred = model.predict_generator(test_generator,steps=186/100) #186 are total example in test set and 40 is batch size
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_true, y_pred,target_names=class_label))


              precision    recall  f1-score   support

Bulging_Eyes       0.28      0.26      0.27        43
   Cataracts       0.04      0.09      0.06        11
Crossed_Eyes       0.55      0.33      0.41        88
    Glaucoma       0.15      0.23      0.18        22
     Uveitis       0.10      0.19      0.13        16

    accuracy                           0.27       180
   macro avg       0.22      0.22      0.21       180
weighted avg       0.36      0.27      0.30       180



In [39]:
K.clear_session()